In [2]:
#https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
df = pd.read_csv('../input/us-accidents/US_Accidents_Dec21_updated.csv')

In [5]:
df.Start_Time

In [6]:
df.info()

# CLEANING WEATHER DATA

In [7]:
df.isnull().sum()

In [8]:
# potencially relevant columns null rates

df[['Visibility(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition']].isnull().sum()/df.shape[0]

In [9]:
# wind and precipitation cannot be used due to too much nulls, other cols are around 2% so we'll be able to dropna if needed

In [10]:
# Severity is not very informative
df.Severity.value_counts(dropna=False, normalize=True)

In [12]:
df['datetime'] = pd.to_datetime(df.Start_Time)
df['year'] = df.datetime.dt.year
df['month'] = df.datetime.dt.month
df['day'] = df.datetime.dt.day
df['dayweek'] = df.datetime.dt.dayofweek
df['yearmonth'] = df.datetime.dt.strftime('%Y-%m')
df['hour'] = df.datetime.dt.hour
df['yearmonth'] = df.datetime.dt.strftime('%Y-%m')

In [13]:
df.iloc[:, -6:].sample(10)

# Accidents overall

In [14]:
df.groupby('year').size().plot(kind='bar')
plt.show()

# Dynamic Severity

In [15]:
df.groupby(['year', 'Severity']).size().reset_index(name='counts').pivot(index='year', columns='Severity', values='counts').plot(kind='bar')
plt.show()

# States by dayweek

In [16]:
sns.set(rc={'figure.figsize':(9,7)})
y_labels = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

In [17]:
#Same code in more explicit way

#how many n largest states we want to analize
n = 10
my_states = df.groupby('State').size().nlargest(n).index

# WHAT YEAR
y = [2021]

dfw = df[df.year.isin(y)].groupby(['State','dayweek']).size().reset_index(name='counts').copy(deep=True) # FILTERING BY YEAR


dfw = dfw.loc[dfw.State.isin(my_states)]


dfw = dfw.pivot(index='dayweek', columns='State', values='counts')
dfw = dfw.apply(lambda x: x/x.sum())
dfw

In [18]:
sns.heatmap(dfw, cmap='rocket_r', yticklabels=y_labels)
plt.title('States compared by day of week accident rates')
plt.show()

In [19]:
# #what state to analize
# s = 'MN'
# #how many n largest cities we want to analize
# n = 10
# labels = df.loc[(df.State == s) & (df.year == y)].groupby('City').size().nlargest(n).index
# df.loc[(df.State == s) & (df.year == y)].groupby('City').size().nlargest(n).plot(kind='pie', labels=labels, labeldistance=0.7, figsize=(8,8))
# plt.title('City contribution')
# plt.ylabel('')
# plt.show()

In [20]:
# dfm = df.loc[(df.State == s) & (df.year == y)].copy(deep=True) # FILTER BY STATE AND YEAR
# my_cities = dfm.groupby('City').size().nlargest(n).index
# my_cities.to_list()

# dfm = dfm.groupby(['City', 'dayweek']).size().reset_index(name='counts')
# dfm = dfm[dfm.City.isin(my_cities)]
# dfm = dfm.pivot(index='dayweek', columns='City', values='counts')#.sort_values()
# dfm = dfm[my_cities] #sorting
# dfm = dfm.apply(lambda x: x/x.sum())
# dfm

In [21]:
# sns.heatmap(dfm, cmap='rocket_r', yticklabels=y_labels)
# plt.show()

# For Tableau with Year - Hour

In [42]:
my_years = [2018, 2019, 2020, 2021]

dft = df[df.year.isin(my_years)].groupby(['State','City','year','month', 'yearmonth','dayweek', 'day','hour']).size().reset_index(name='counts').copy(deep=True) # for Tableau

In [43]:
dft.sample(10)

# Population

In [24]:
dfp = pd.read_csv('../input/us-2021-census-cities-populations-coordinates/us2021census.csv')

dfp.sort_values(by='Population', ascending=False).head()

In [25]:
dft['State-City'] = df.State.str.cat(df.City, sep='-')
dfm = dft.loc[dft.year == 2020, ['City', 'counts']].groupby('City').sum().merge(dfp, left_index=True, right_on='City')
dfm.head()

## Month-Day relationship

In [26]:
dft.head()

In [27]:
#dft.to_csv('accidents_us_hour2.csv')

In [62]:
my_st = 'FL'
my_yr = 2020
df_day_month = dft[(dft.State == my_st) & (dft.year == my_yr)].groupby(['month', 'day'], as_index=False).counts.sum()
df_day_month['month'] = df_day_month.month.astype('category')
df_day_month['day'] = df_day_month.day.astype('category')

In [63]:
#plt.figure(figsize=(45,10))
sns.set_style('white')
g = sns.relplot(x='day', y='counts', data = df_day_month , kind='line', col='month',  col_wrap=4) # style='month', hue='month', 

plt.show()

## State-Month

In [30]:
dft.groupby('State').counts.sum().sort_values(ascending=False).head(8).index

In [52]:
#my_yr = 2019
n_states = 8 # how many states to show
my_sts = dft.groupby('State').counts.sum().sort_values(ascending=False).head(n_states).index # ['CA', 'TX', '']
#df_state_month = dft[(dft.State.isin(my_sts)) & (dft.year == my_yr)].groupby(['State', 'month'], as_index=False).counts.sum()
df_state_month = dft[dft.State.isin(my_sts)].groupby(['State', 'year','month'], as_index=False).counts.sum()
#df_day_month['month'] = df_day_month.month.astype('category')
#df_day_month['day'] = df_day_month.day.astype('category')

In [44]:
n_states = 8 # how many states to show
my_sts = dft.groupby('State').counts.sum().sort_values(ascending=False).head(n_states).index # ['CA', 'TX', '']
#df_state_month = dft[(dft.State.isin(my_sts)) & (dft.year == my_yr)].groupby(['State', 'month'], as_index=False).counts.sum()

df_state_month = dft[dft.State.isin(my_sts)].groupby(['State','yearmonth'], as_index=False).counts.sum()

In [59]:
sns.set_palette('RdBu')
g = sns.relplot(x='month', y='counts', data = df_state_month , kind='line', hue='State', col='year')#, height=7, aspect=1.6) 

g.set(xlabel='month', ylabel='number of accindents')
g.fig.suptitle('Number of accidents by year, month and state', y=1.08)
plt.xticks(rotation=90)
plt.show()

In [33]:
type(g)

In [34]:
dfm.sort_values(by='Population', ascending=False)

In [35]:
dfm

In [36]:
dfm[dfm.City == 'Wilmington']

In [37]:
dfm[dfm.State.str.cat(dfm.City, sep='-').duplicated() == True].sort_values(['State', 'City']).head(20)

In [38]:
df.dtypes

In [39]:
plt.hist(df['Visibility(mi)'], bins=100)
plt.show()

In [40]:
df['Visibility(mi)'].dropna().sort_values(ascending=False).head(30)

In [41]:
df.groupby('Visibility(mi)').size().sort_values(ascending=False)